In [1]:
# Libraries
import tensorflow
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, ConcatDataset
from transformers import AutoTokenizer

import pandas as pd
import datetime
import time
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sn
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import pickle
from itertools import product

from ABSA_SentimentMultiEmiten.model.bert import bert_ABSA
from ABSA_SentimentMultiEmiten.data.dataset import dataset_ABSA

2023-06-02 06:41:30.843529: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Menentukan device yang akan digunakan untuk melakukan komputasi
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Tensorflow\t: ", tensorflow.__version__)
print("Torch\t\t: ", torch.__version__)
print("Device\t\t: ", DEVICE)
print("GPU\t\t: ", torch.cuda.get_device_name())
print("CUDA\t\t: ", torch.version.cuda)

Tensorflow	:  2.4.1
Torch		:  1.4.0
Device		:  cuda:0
GPU		:  Tesla T4
CUDA		:  10.1


In [3]:
# Inisialisasi pre-trained model IndoBERT
pretrained_model_name = "indobenchmark/indobert-base-p1"
model_name = "indonlu-indobert-gs"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
model_ABSA = bert_ABSA(pretrained_model_name)
model_ABSA.to(DEVICE)

bert_ABSA(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [4]:
# Function untuk konversi waktu dari detik ke jam, menit, detik
def evl_time(t):
    min, sec= divmod(t, 60)
    hr, min = divmod(min, 60)
    return int(hr), int(min), int(sec)

# Function untuk menyimpan model
def save_model(model, name):
    torch.save(model.state_dict(), name)

# Function untuk load model
def load_model(model, path):
    model.load_state_dict(torch.load(path), strict=False)
    return model

In [5]:
# Mengubah dataset menjadi beberapa mini-batch
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    segments_tensors = [s[2] for s in samples]
    segments_tensors = pad_sequence(segments_tensors, batch_first=True)

    label_ids = torch.stack([s[3] for s in samples])
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)

    return ids_tensors, segments_tensors, masks_tensors, label_ids

In [6]:
# Function untuk pelatihan model
def train_model_ABSA(loader, epochs, model_name):
    history = {'loss' : []}
    all_data = len(loader)
    
    # Pengulangan epoch
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []
        
        # Pengulangan setiap mini-batch
        for data in loader:
            t0 = time.time()
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            label_ids = label_ids.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            loss = model_ABSA(ids_tensors=ids_tensors, lable_tensors=label_ids, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            losses.append(loss.item())
            loss.backward()
            optimizer_ABSA.step()
            optimizer_ABSA.zero_grad()

            finish_data += 1
            current_times.append(round(time.time()-t0,3))
            current = np.mean(current_times)
            hr, min, sec = evl_time(current*(all_data-finish_data) + current*all_data*(epochs-epoch-1))
            print('epoch:', epoch+1, "/" , epochs," batch:", finish_data, "/" , all_data, " loss:", np.mean(losses), " hr:", hr, " min:", min," sec:", sec)

        history['loss'].append(np.mean(losses))
        save_model(model_ABSA, model_name)
    
    return history

# Function untuk pengujian model
def test_model_ABSA(loader):
    pred = []
    truth = []
    
    with torch.no_grad():
        # Pengulangan setiap mini-batch
        for data in loader:
            ids_tensors, segments_tensors, masks_tensors, label_ids = data
            ids_tensors = ids_tensors.to(DEVICE)
            segments_tensors = segments_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ABSA(ids_tensors, None, masks_tensors=masks_tensors, segments_tensors=segments_tensors)
            
            _, predictions = torch.max(outputs, dim=1)

            pred += list([int(i) for i in predictions])
            truth += list([int(i) for i in label_ids])

    return truth, pred

In [7]:
def record_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_timedelta = datetime.timedelta(seconds=elapsed_time)

    str_obj = datetime.datetime.fromtimestamp(start_time)
    end_obj = datetime.datetime.fromtimestamp(end_time)

    time_str = str_obj.strftime("%d/%m/%y %H:%M:%S")
    time_end = end_obj.strftime("%d/%m/%y %H:%M:%S")

    return str(elapsed_timedelta)

# Function menghitung dua waktu dan menyimpan ke dalam file
def record_time_to_file(start_time, end_time, file):
    elapsed_time = end_time - start_time
    elapsed_timedelta = datetime.timedelta(seconds=elapsed_time)

    str_obj = datetime.datetime.fromtimestamp(start_time)
    end_obj = datetime.datetime.fromtimestamp(end_time)

    time_str = str_obj.strftime("%d/%m/%y %H:%M:%S")
    time_end = end_obj.strftime("%d/%m/%y %H:%M:%S")

    with open(file, "w") as f:
        f.write("Waktu mulai: " + time_str)
        f.write("\nWaktu selesai: " + time_end)
        f.write("\nWaktu total: " + str(elapsed_timedelta))

In [8]:
# Inisialisasi dataset
emiten_train_ds = dataset_ABSA(pd.read_csv("data_experiment/data_balance_experiment_training.csv"), tokenizer)
emiten_test_ds = dataset_ABSA(pd.read_csv("data_experiment/data_balance_experiment_testing.csv"), tokenizer)

train_ds = ConcatDataset([emiten_train_ds])
test_ds = ConcatDataset([emiten_test_ds])

In [9]:
# Inisialisasi hyperparameter space
bs_list = [16, 32]
lr_list = [0.00002, 0.00003, 0.00005]
epoch_list = [5, 25, 50]

In [ ]:
# Inisialisasi variabel untuk menyimpan hasil
path = "indonlu-indobert-gs/"
indonlu_result = []
j = 0

# Waktu mulai
start_time = time.time()

# Grid Search
for bs, lr, epoch in product(bs_list, lr_list, epoch_list):
    # Inisialisasi pretrained-model
    model_ABSA = bert_ABSA(pretrained_model_name)
    model_ABSA.to(DEVICE)
    
    print(f'Training with batch size={bs}, learning rate={lr}, epoch={epoch}')

    optimizer_ABSA = torch.optim.Adam(model_ABSA.parameters(), lr=lr)

    # Mengubah dataset menjadi mini-batch
    train_loader = DataLoader(train_ds, batch_size=bs, collate_fn=create_mini_batch, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=bs, collate_fn=create_mini_batch, shuffle=True)
    
    # Waktu mulai training model
    model_start = time.time()
    
    # Train model
    history = %time train_model_ABSA(train_loader, epoch, (path+model_name+'-'+str(j)+'.pkl'))
    model_end = time.time()
    
    # Classification report
    model_ABSA = load_model(model_ABSA, (path+model_name+'-'+str(j)+'.pkl'))
    x, y = test_model_ABSA(test_loader)
    report = classification_report(x, y, target_names=[str(i) for i in range(3)])
    
    # Simpan hasil ke dalam variabel
    result = {'bs': bs, 'lr': lr, 'epoch': epoch, 'history': history, 'report': report,
              'name': (model_name+'-'+str(j)+'.pkl'), 'time': record_time(model_start, model_end)}
    indonlu_result.append(result)
    
    j+=1
    
# Waktu selesai
end_time = time.time()  
record_time_to_file(start_time, end_time, path+'waktu_indonlu_total.txt')

# Simpan hasil ke dalam file
with open(path+'indonlu_gs_result.pkl', 'wb') as f:
    pickle.dump(indonlu_result, f)

Training with batch size=16, learning rate=2e-05, epoch=5
epoch: 1 / 5  batch: 1 / 612  loss: 1.093851923942566  hr: 0  min: 32  sec: 56
epoch: 1 / 5  batch: 2 / 612  loss: 1.2182469367980957  hr: 0  min: 28  sec: 8
epoch: 1 / 5  batch: 3 / 612  loss: 1.1479264895121257  hr: 0  min: 25  sec: 28
epoch: 1 / 5  batch: 4 / 612  loss: 1.1574043035507202  hr: 0  min: 24  sec: 5
epoch: 1 / 5  batch: 5 / 612  loss: 1.1507184982299805  hr: 0  min: 24  sec: 8
epoch: 1 / 5  batch: 6 / 612  loss: 1.1296786864598591  hr: 0  min: 21  sec: 32
epoch: 1 / 5  batch: 7 / 612  loss: 1.111797673361642  hr: 0  min: 21  sec: 11
epoch: 1 / 5  batch: 8 / 612  loss: 1.1055898666381836  hr: 0  min: 20  sec: 57
epoch: 1 / 5  batch: 9 / 612  loss: 1.1273527145385742  hr: 0  min: 20  sec: 52
epoch: 1 / 5  batch: 10 / 612  loss: 1.122122049331665  hr: 0  min: 21  sec: 1
epoch: 1 / 5  batch: 11 / 612  loss: 1.1385251717133955  hr: 0  min: 21  sec: 48
epoch: 1 / 5  batch: 12 / 612  loss: 1.114122062921524  hr: 0  min:

epoch: 1 / 5  batch: 102 / 612  loss: 0.8471948936873791  hr: 0  min: 19  sec: 25
epoch: 1 / 5  batch: 103 / 612  loss: 0.8441375725477644  hr: 0  min: 19  sec: 25
epoch: 1 / 5  batch: 104 / 612  loss: 0.8424207407694596  hr: 0  min: 19  sec: 26
epoch: 1 / 5  batch: 105 / 612  loss: 0.8406910992804028  hr: 0  min: 19  sec: 26
epoch: 1 / 5  batch: 106 / 612  loss: 0.8376577985736559  hr: 0  min: 19  sec: 28
epoch: 1 / 5  batch: 107 / 612  loss: 0.8337931869743026  hr: 0  min: 19  sec: 23
epoch: 1 / 5  batch: 108 / 612  loss: 0.8345972114691028  hr: 0  min: 19  sec: 18
epoch: 1 / 5  batch: 109 / 612  loss: 0.8318852522504439  hr: 0  min: 19  sec: 17
epoch: 1 / 5  batch: 110 / 612  loss: 0.8303080442276868  hr: 0  min: 19  sec: 22
epoch: 1 / 5  batch: 111 / 612  loss: 0.828533810806704  hr: 0  min: 19  sec: 21
epoch: 1 / 5  batch: 112 / 612  loss: 0.8240487554243633  hr: 0  min: 19  sec: 20
epoch: 1 / 5  batch: 113 / 612  loss: 0.8222943375595902  hr: 0  min: 19  sec: 21
epoch: 1 / 5  bat

epoch: 1 / 5  batch: 203 / 612  loss: 0.7061531567955251  hr: 0  min: 19  sec: 8
epoch: 1 / 5  batch: 204 / 612  loss: 0.7049591574160492  hr: 0  min: 19  sec: 8
epoch: 1 / 5  batch: 205 / 612  loss: 0.7057381787678091  hr: 0  min: 19  sec: 4
epoch: 1 / 5  batch: 206 / 612  loss: 0.7055013981982342  hr: 0  min: 19  sec: 6
epoch: 1 / 5  batch: 207 / 612  loss: 0.7054986908383991  hr: 0  min: 19  sec: 8
epoch: 1 / 5  batch: 208 / 612  loss: 0.7065226401990423  hr: 0  min: 19  sec: 10
epoch: 1 / 5  batch: 209 / 612  loss: 0.7058115456805846  hr: 0  min: 19  sec: 10
epoch: 1 / 5  batch: 210 / 612  loss: 0.7044918586100851  hr: 0  min: 19  sec: 9
epoch: 1 / 5  batch: 211 / 612  loss: 0.7040804218483198  hr: 0  min: 19  sec: 9
epoch: 1 / 5  batch: 212 / 612  loss: 0.70367513857079  hr: 0  min: 19  sec: 9
epoch: 1 / 5  batch: 213 / 612  loss: 0.7021707425794691  hr: 0  min: 19  sec: 9
epoch: 1 / 5  batch: 214 / 612  loss: 0.7013649815850169  hr: 0  min: 19  sec: 9
epoch: 1 / 5  batch: 215 / 6

epoch: 1 / 5  batch: 304 / 612  loss: 0.6429821751325538  hr: 0  min: 18  sec: 38
epoch: 1 / 5  batch: 305 / 612  loss: 0.6420647346094006  hr: 0  min: 18  sec: 36
epoch: 1 / 5  batch: 306 / 612  loss: 0.6417870037014188  hr: 0  min: 18  sec: 34
epoch: 1 / 5  batch: 307 / 612  loss: 0.6412384478490594  hr: 0  min: 18  sec: 33
epoch: 1 / 5  batch: 308 / 612  loss: 0.6405983113913567  hr: 0  min: 18  sec: 33
epoch: 1 / 5  batch: 309 / 612  loss: 0.6405360686643996  hr: 0  min: 18  sec: 32
epoch: 1 / 5  batch: 310 / 612  loss: 0.6405554215273549  hr: 0  min: 18  sec: 32
epoch: 1 / 5  batch: 311 / 612  loss: 0.6404192960818097  hr: 0  min: 18  sec: 32
epoch: 1 / 5  batch: 312 / 612  loss: 0.6404247250981056  hr: 0  min: 18  sec: 32
epoch: 1 / 5  batch: 313 / 612  loss: 0.6397671162034757  hr: 0  min: 18  sec: 31
epoch: 1 / 5  batch: 314 / 612  loss: 0.639377332369613  hr: 0  min: 18  sec: 31
epoch: 1 / 5  batch: 315 / 612  loss: 0.6390455981568685  hr: 0  min: 18  sec: 31
epoch: 1 / 5  bat

epoch: 1 / 5  batch: 406 / 612  loss: 0.5985530709545014  hr: 0  min: 18  sec: 2
epoch: 1 / 5  batch: 407 / 612  loss: 0.5974763503678014  hr: 0  min: 18  sec: 1
epoch: 1 / 5  batch: 408 / 612  loss: 0.5973653212627944  hr: 0  min: 18  sec: 2
epoch: 1 / 5  batch: 409 / 612  loss: 0.5974007498809936  hr: 0  min: 18  sec: 1
epoch: 1 / 5  batch: 410 / 612  loss: 0.5966685936945241  hr: 0  min: 18  sec: 1
epoch: 1 / 5  batch: 411 / 612  loss: 0.5959854509563631  hr: 0  min: 18  sec: 1
epoch: 1 / 5  batch: 412 / 612  loss: 0.5952186193136335  hr: 0  min: 18  sec: 0
epoch: 1 / 5  batch: 413 / 612  loss: 0.5948948095987842  hr: 0  min: 18  sec: 0
epoch: 1 / 5  batch: 414 / 612  loss: 0.5942434470578668  hr: 0  min: 18  sec: 0
epoch: 1 / 5  batch: 415 / 612  loss: 0.5938501266829939  hr: 0  min: 17  sec: 59
epoch: 1 / 5  batch: 416 / 612  loss: 0.5939097919573004  hr: 0  min: 17  sec: 59
epoch: 1 / 5  batch: 417 / 612  loss: 0.5933108549180934  hr: 0  min: 17  sec: 58
epoch: 1 / 5  batch: 418 

...

epoch: 6 / 25  batch: 279 / 612  loss: 0.09573913278812576  hr: 1  min: 26  sec: 59
epoch: 6 / 25  batch: 280 / 612  loss: 0.09544965775151337  hr: 1  min: 27  sec: 1
epoch: 6 / 25  batch: 281 / 612  loss: 0.09514177557465445  hr: 1  min: 27  sec: 5
epoch: 6 / 25  batch: 282 / 612  loss: 0.09511621281874517  hr: 1  min: 27  sec: 4
epoch: 6 / 25  batch: 283 / 612  loss: 0.09478995036794524  hr: 1  min: 26  sec: 55
epoch: 6 / 25  batch: 284 / 612  loss: 0.09597500121142243  hr: 1  min: 26  sec: 56
epoch: 6 / 25  batch: 285 / 612  loss: 0.09718850779167393  hr: 1  min: 26  sec: 56
epoch: 6 / 25  batch: 286 / 612  loss: 0.09809150647085446  hr: 1  min: 26  sec: 57
epoch: 6 / 25  batch: 287 / 612  loss: 0.09778840046778373  hr: 1  min: 26  sec: 48
epoch: 6 / 25  batch: 288 / 612  loss: 0.09922922124517047  hr: 1  min: 26  sec: 47
epoch: 6 / 25  batch: 289 / 612  loss: 0.09942476210348747  hr: 1  min: 26  sec: 46
epoch: 6 / 25  batch: 290 / 612  loss: 0.09967083639368929  hr: 1  min: 26  sec

epoch: 6 / 25  batch: 377 / 612  loss: 0.10461390971030733  hr: 1  min: 26  sec: 13
epoch: 6 / 25  batch: 378 / 612  loss: 0.10451743259001031  hr: 1  min: 26  sec: 12
epoch: 6 / 25  batch: 379 / 612  loss: 0.10484278960875912  hr: 1  min: 26  sec: 15
epoch: 6 / 25  batch: 380 / 612  loss: 0.10483587107768184  hr: 1  min: 26  sec: 17
epoch: 6 / 25  batch: 381 / 612  loss: 0.10480446354767156  hr: 1  min: 26  sec: 16
epoch: 6 / 25  batch: 382 / 612  loss: 0.10499373870214243  hr: 1  min: 26  sec: 8
epoch: 6 / 25  batch: 383 / 612  loss: 0.10493026809966284  hr: 1  min: 26  sec: 3
epoch: 6 / 25  batch: 384 / 612  loss: 0.105883179542919  hr: 1  min: 26  sec: 2
epoch: 6 / 25  batch: 385 / 612  loss: 0.10562938788494507  hr: 1  min: 26  sec: 3
epoch: 6 / 25  batch: 386 / 612  loss: 0.10537921417253623  hr: 1  min: 26  sec: 2
epoch: 6 / 25  batch: 387 / 612  loss: 0.1052469594611062  hr: 1  min: 26  sec: 1
epoch: 6 / 25  batch: 388 / 612  loss: 0.10506829481149457  hr: 1  min: 26  sec: 1
ep

epoch: 6 / 25  batch: 476 / 612  loss: 0.11175767671974267  hr: 1  min: 25  sec: 18
epoch: 6 / 25  batch: 477 / 612  loss: 0.11156364828660553  hr: 1  min: 25  sec: 12
epoch: 6 / 25  batch: 478 / 612  loss: 0.11148686663376238  hr: 1  min: 25  sec: 11
epoch: 6 / 25  batch: 479 / 612  loss: 0.11176953693396861  hr: 1  min: 25  sec: 10
epoch: 6 / 25  batch: 480 / 612  loss: 0.11237615430727602  hr: 1  min: 25  sec: 9
epoch: 6 / 25  batch: 481 / 612  loss: 0.11230990667043257  hr: 1  min: 25  sec: 8
epoch: 6 / 25  batch: 482 / 612  loss: 0.11221506418776216  hr: 1  min: 25  sec: 7
epoch: 6 / 25  batch: 483 / 612  loss: 0.112455182502482  hr: 1  min: 25  sec: 9
epoch: 6 / 25  batch: 484 / 612  loss: 0.11243989136962851  hr: 1  min: 25  sec: 9
epoch: 6 / 25  batch: 485 / 612  loss: 0.11226532030658623  hr: 1  min: 25  sec: 10
epoch: 6 / 25  batch: 486 / 612  loss: 0.11212519100970691  hr: 1  min: 25  sec: 10
epoch: 6 / 25  batch: 487 / 612  loss: 0.11204266600173111  hr: 1  min: 25  sec: 15

epoch: 6 / 25  batch: 574 / 612  loss: 0.11720905484118943  hr: 1  min: 24  sec: 21
epoch: 6 / 25  batch: 575 / 612  loss: 0.11768267152102098  hr: 1  min: 24  sec: 22
epoch: 6 / 25  batch: 576 / 612  loss: 0.11761842268363883  hr: 1  min: 24  sec: 21
epoch: 6 / 25  batch: 577 / 612  loss: 0.11749517255215752  hr: 1  min: 24  sec: 20
epoch: 6 / 25  batch: 578 / 612  loss: 0.11786870966475316  hr: 1  min: 24  sec: 19
epoch: 6 / 25  batch: 579 / 612  loss: 0.11781298717892437  hr: 1  min: 24  sec: 19
epoch: 6 / 25  batch: 580 / 612  loss: 0.1178799988894627  hr: 1  min: 24  sec: 18
epoch: 6 / 25  batch: 581 / 612  loss: 0.11777042455188994  hr: 1  min: 24  sec: 17
epoch: 6 / 25  batch: 582 / 612  loss: 0.11787704795701397  hr: 1  min: 24  sec: 16
epoch: 6 / 25  batch: 583 / 612  loss: 0.11810814495970004  hr: 1  min: 24  sec: 11
epoch: 6 / 25  batch: 584 / 612  loss: 0.11792850354048487  hr: 1  min: 24  sec: 17
epoch: 6 / 25  batch: 585 / 612  loss: 0.1178441232850409  hr: 1  min: 24  se

epoch: 7 / 25  batch: 61 / 612  loss: 0.08199381755023706  hr: 1  min: 22  sec: 15
epoch: 7 / 25  batch: 62 / 612  loss: 0.08132586411891445  hr: 1  min: 21  sec: 32
epoch: 7 / 25  batch: 63 / 612  loss: 0.08342549725184364  hr: 1  min: 22  sec: 10
epoch: 7 / 25  batch: 64 / 612  loss: 0.08299713849555701  hr: 1  min: 21  sec: 32
epoch: 7 / 25  batch: 65 / 612  loss: 0.08188394182003461  hr: 1  min: 21  sec: 45
epoch: 7 / 25  batch: 66 / 612  loss: 0.08862176955197797  hr: 1  min: 22  sec: 18
epoch: 7 / 25  batch: 67 / 612  loss: 0.08934767977960074  hr: 1  min: 22  sec: 24
epoch: 7 / 25  batch: 68 / 612  loss: 0.08907240870244362  hr: 1  min: 22  sec: 25
epoch: 7 / 25  batch: 69 / 612  loss: 0.0901891137810721  hr: 1  min: 22  sec: 23
epoch: 7 / 25  batch: 70 / 612  loss: 0.0904484038906438  hr: 1  min: 22  sec: 27
epoch: 7 / 25  batch: 71 / 612  loss: 0.09027629545037176  hr: 1  min: 22  sec: 57
epoch: 7 / 25  batch: 72 / 612  loss: 0.09175894657770793  hr: 1  min: 23  sec: 6
epoch: 